In [2]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 999

In [3]:
#Read data
df = pd.read_csv("../../Data/Tidy_Full.csv", index_col = 0)

In [4]:
#Inspect data
df.head()

,CompID,Name,Sex,Class,Division,BestBP,Total,Place,BestSQ,BestDL,...,BP2,BP3,Wilks,SQ1,SQ2,SQ3,DL1,DL2,DL3,DOB
Index,,,,,,,,,,,,,,,,,,,,,
0,1,Barbara Hamilton,F,56,O/50,42.5/43,42.5,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Nat Zacharko,F,56,O/40,60,60,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Dean Jenkinson,M,67.5,Open,125,125,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Aaron McFarlane,M,67.5,Open,105,105,2-Jan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,David Graham,M,67.5,O/60,52.5,52.5,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Goals:
- Homogenize column values
- Fix data types
- Change no-lifts to minus (rather than x suffix)

##### Sex column

In [5]:
#Inspect
df.Sex.value_counts()

M           15488
F            6969
M-O            63
M-J            43
m              42
F-O            18
F-J            17
f              16
M-M1           16
M - Open       12
M-SJ           11
M-M3           11
F-M1           11
F - M1          7
F - Open        6
M - O           6
M-M2            6
M-Sub           5
W               4
M - M2          3
M-BL            2
F - M2          2
F-SJ            2
F-M3            2
F-M4            1
M - U18         1
M-Jnr           1
M - M1          1
F-M2            1
M - SJ          1
Name: Sex, dtype: int64

In [6]:
#Fix
def fix_sex(df):
    df.Sex = df.Sex.replace({'m': 'M', 'f': 'F', 'W': 'F'})
    df.Sex = df.Sex.str.replace(' ','')
    df[df.Sex.str.contains('-')].Division.isnull().all() #All rows with div info in sex column have nulls in div column
    df.Division.fillna(pd.DataFrame(df.Sex.str.split('-').to_list(), columns = ['Sex', 'Division'])['Division'], inplace = True)
    df.Sex = pd.DataFrame(df.Sex.str.split('-').to_list(), columns = ['Sex', 'Division'])['Sex']
    return df

##### Class column

In [7]:
#Inspect
#df[~df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts() #Possibly bad values
df[df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts() #Okay values
#df[df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts().plot(kind='bar') #Histogram of okay values

82.5      846
90        767
75        735
100       598
67.5      504
83        483
93        473
110       459
105       447
125       390
74        315
120       261
125+      238
60        228
56        173
120+      167
66        166
52        162
63        145
72        121
57        110
84         91
84+        90
90+        85
48         56
59         42
47         37
44          5
75.5        1
92.54       1
55.9        1
120.8       1
89.2        1
102         1
76.4        1
67.6        1
117.26      1
66.9        1
89.6        1
62.5        1
Name: Class, dtype: int64

In [8]:
#Fix
def fix_class(df):
    df.Class = df.Class.str.replace('kg', '')
    df.Class = df.Class.str.replace('x', '')
    df.Class = df.Class.str.replace(' ', '')
    df[df.Class.fillna('').str.contains('-')].Division.isnull().all() #All rows with div info in class column have nulls in div column
    df.Division.fillna(pd.DataFrame(df.Class.fillna('').str.split('-').to_list(), columns = ['Class', 'Division'])['Division'], inplace = True)
    df.Class = pd.DataFrame(df.Class.fillna('').str.split('-').to_list(), columns = ['Class', 'Division'])['Class']
    df.Class.replace('', np.nan, inplace = True)
    #Look up special case comp tables to check for errors
    df.replace({'Class' : { '125.00+' : '125+', '66.9': '67.5', '55.9': '56', '62.5': '67.5', '67.6': '67.5', '75.5': '75'}}, inplace = True)
    for i in [1338, 1556, 1558, 3446, 3473, 7510, 7528]:
        temp = df.loc[i, 'BDW']
        df.loc[i, 'BDW'] = df.loc[i, 'Class']
        df.loc[i, 'Class'] = temp
    return df

##### DOB column

In [9]:
#Inspect
df.DOB.value_counts()
#set([k[0] for j, k in df[df.DOB.notnull()].iterrows() if any(substring in k[21] for substring in ['PNG', 'NRU', '????', 'DQ', '-', 'NAU', 'NZPF', 'TUV', '???', 'PRC', 'NZ', 'NIU'])])

1993     942
1992     813
1994     811
1995     772
1991     626
1989     619
1990     616
1996     614
1997     545
1987     491
1988     465
1986     364
1998     356
1982     344
1985     317
1984     305
1981     292
1983     277
1980     256
1999     236
1979     204
2000     201
1977     187
1975     166
1978     151
1970     149
1973     149
1974     142
2001     136
1976     126
2002     123
1969     122
1965     117
1966     113
1971     106
1967     102
1968     102
1972     101
1964      95
1961      90
1962      85
1956      79
1948      74
OPEN      69
1963      65
1958      62
1960      53
2003      52
1959      46
1957      42
1955      41
1949      37
2004      35
1945      32
1954      30
1950      29
1947      29
1953      28
O/40      27
1952      26
23        24
U/23      23
O         22
28        20
1944      20
O/60      19
1951      16
0         16
25        15
O/50      15
1939      15
27        14
26        14
PNG       13
NRU       12
35        11
1941      11

In [10]:
#Fix
def fix_dob(df):
    #Division in DOB column
    for i in [103, 109, 113, 114, 141, 142, 144, 146, 168, 177, 184, 214]:
        df.loc[df.CompID == i, 'Division'] = df.loc[df.CompID == i, 'DOB']
        df.loc[df.CompID == i, 'DOB'] = np.nan
    #Words in DOB column
    df.DOB.replace(['PNG', 'NRU', '????', 'DQ', '-', 'NAU', 'NZPF', 'TUV', '???', 'PRC', 'NZ', 'NIU'], np.nan, inplace = True)
    df.DOB.replace('`987', 1987, inplace = True)
    #Big numbers
    df.DOB.replace('\d{5}', np.nan, inplace = True, regex = True)
    #Small numbers
    df.replace({'DOB' : { '0' : np.nan, '1886': 1986, '1680': 1980, '995': 1995}}, inplace = True)
    df.loc[17261, 'DOB'] = 2018-30; df.loc[12972, 'DOB'] = 2016-35; df.loc[12975, 'DOB'] = 2018-30; df.loc[18635, 'DOB'] = 2018-27
    for i,j in zip([572, 574, 606, 612, 665, 939, 940], [2015, 2015, 2016, 2016, 2017, 2020, 2020]):
        df.loc[df.CompID == i, 'DOB'] = j-df.loc[df.CompID == i, 'DOB']
    #Too close to current year
    df.replace({'DOB' : { 2014 : np.nan, 2016: np.nan, 2020: np.nan}}, inplace = True)
    df.loc[12166, 'DOB'] = np.nan
    return df

##### Division column

In [11]:
#Inspect
df.Division.value_counts()
#Check rows with weird entries
#set([k[0] for j, k in df[df.Division.notnull()].iterrows() if any(substring in k[4] for substring in ['"'])])

O              1887
Open            681
J               503
M1              498
M2              310
SJ              212
M3              192
O/40            160
OPEN            131
"               114
O/50             90
O/60             89
U/23             80
Jnr              54
M4               50
VIC              35
Op               31
JNR              30
U18              26
o                23
U/19             21
BP               20
Classic          20
Jun              20
MI               18
Bp               17
O-O              17
Junior           17
U23              16
Nov              15
open             15
SJNR             15
SUB JNR          13
O-O/40           12
U/18             11
M                10
JR                9
0/50              9
O/70              8
Guest             8
MII               8
b                 8
0/40              7
OVER 40           7
MIII              7
Uni               7
OPEN?             7
Op-O/40           6
APFC              6
Oceania           6


In [12]:
#Fix
def fix_division(df):
    df.loc[df.CompID == 64, 'Division'] = df[df.CompID == 64].Division.str.replace('"', 'Snr')
    df.Division = df.Division.str.replace('"', 'Open')
    divsemdf = pd.read_csv("../../Data/Division_semantics.csv", engine = 'python')
    #Map words to semantic meaning
    semdict = {tuple(divsemdf[divsemdf[col].notnull()][col]): col for col in divsemdf.columns}
    semmap = {}
    for k, v in semdict.items():
        for key in k:
            semmap[key] = v
    #Change words
    df.Division.replace(semmap, inplace = True)
    df.Division.replace('Unknown', np.nan, inplace = True)
    df.loc[[5546, 3277], 'Division'] = 'MI'
    return df

##### Wilks column

In [13]:
#Inspect
#df.Wilks.value_counts()
df[~df.Wilks.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Wilks.value_counts()
#df[df.Wilks.str.match('\.$') == True] #Check none start with .
#df[df.Wilks.str.match('^\.') == True] #Check none end with .

BMB          389
-            368
x              6
out            5
Disq.          4
344,24         2
264,79         2
369,65         2
297,47         2
397,16         2
284,38         2
324,69         2
358,17         2
423,08         2
423,41         2
375,98         2
411,60         2
408,68         2
345,10         2
449,17         2
265,36         2
343,02         2
450,14         2
275,57         2
358,68         2
418,95         2
340,41         2
352,01         2
350,47         2
309,60         2
318,96         2
387,15         2
361,59         2
369,20         2
300,51         2
324,09         2
344,52         2
312,56         2
337,26         2
281,74         2
356,67         2
231,37         2
296,59         2
338,72         2
331,04         2
373,91         2
331,55         2
314,68         1
324,95         1
287,74         1
292,58         1
266,77         1
319,52         1
422,25         1
341,44         1
428,21         1
488,03         1
369,71         1
342,67        

In [14]:
#Fix
def fix_wilks(df):
    df.Wilks.replace(['BMB', '-', 'out', 'x', 'Disq.', 'Disq', '?', 'DQ (fees)', 'withdrawn', 'DQ'], np.nan, inplace = True)
    df.Wilks = df.Wilks.str.replace(',', '.', regex = False)
    return df

##### BDW column

In [15]:
#Inspect
#df.BDW.value_counts()
df[~df.BDW.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].BDW.value_counts()

92,20     5
81,85     5
73,85     5
65,75     4
80,80     4
72,90     4
82,95     4
82,10     4
73,80     4
55,00     4
103,90    4
119,40    4
72,00     4
73,70     3
92,00     3
62,90     3
69,70     3
92,45     3
70,70     3
75,90     3
71,60     3
90,80     3
82,25     3
78,85     3
73,20     3
82,00     3
80,60     3
71,00     3
63,00     3
64,90     3
82,50     3
91,25     3
71,40     3
60,30     3
92,65     3
104,25    3
69,90     3
117,70    3
89,55     3
92,80     3
64,00     3
78,50     3
82,90     3
62,30     3
102,95    3
80,30     3
101,55    3
97,50     3
72,70     2
91,10     2
82,75     2
113,25    2
51,40     2
119,15    2
62,95     2
65,65     2
80,20     2
91,75     2
91,60     2
95,60     2
103,40    2
104,65    2
62,85     2
65,50     2
92,90     2
50,95     2
82,20     2
62,50     2
90,00     2
92,60     2
136,60    2
81,50     2
82,40     2
104,90    2
81,30     2
92,55     2
107,90    2
104,55    2
92,50     2
102,35    2
92,70     2
73,75     2
81,20     2
71,7

In [16]:
#Fix
def fix_bdw(df):
    df.BDW = df.BDW.str.replace(',', '.', regex = False)
    df.BDW = df.BDW.str.replace('..', '.', regex = False)
    df.BDW = df.BDW.str.replace('?', '', regex = False)
    df.BDW.replace('BMB', np.nan, inplace = True)
    return df

##### Place column

In [17]:
#Inspect
df.Place.value_counts()
#df[df.Place.fillna('1').str.isnumeric()].Place.value_counts() #Fine
#df[~df.Place.fillna('1').str.isnumeric()].Place.value_counts() #Bad

#Bad categories:
#df[~df.Place.fillna('').str.contains('\d')].Place.value_counts() #No info
#df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].Place.value_counts() #Suffix + Additional info included
#df[df.Place.fillna('').str.match('.+(kg$)')] #All of these have division/class info already - can remove from place column
#df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)$')].Place.value_counts() #Has "th", "st", "nd", "rd" suffix
#df[df.Place.fillna('').str.contains('[a-zA-Z]')].Place.value_counts() #Additional info (from one comp only)
#df[~df.Place.fillna('1').str.isnumeric()].CompID.unique() #Comps with placing for 2 classes

1                7648
2                3566
3                2030
4                1209
5                 726
6                 459
7                 259
8                 172
-                 148
9                 111
10                 74
1st                61
11                 58
?-?                47
12                 44
GOLD               43
2nd                42
-?                 38
BMB                36
1 & 1              30
13                 30
3rd                28
14                 20
4th                20
SILVER             17
15                 16
5th                15
BRONZE             11
16                 11
7th                10
x                   8
19                  8
6th                 8
18                  8
17                  8
12th                7
9th                 7
20                  6
10th                6
1.1                 5
21                  5
22                  5
1 & 2               4
1st J 66kg          4
1st M1 120+kg       4
Disq.     

In [18]:
#Fix
def fix_place(df):
    df.Place.replace(['0', '-', '?-?', '-?', 'BMB', 'x', 'DQ', 'Disq.', '?', 'Inv', '?-??', 'X', 'Guest', 'N.E.', 'Most Imp', 'G', 'Disq', 'bmb'], np.nan, inplace = True)
    df.replace({'Place' : {'GOLD': '1', 'SILVER': '2', 'BRONZE': '3', '`.1': '1', '`1': '1'}}, inplace = True)
    df.Place = df.Place.str.replace('Jan', '1', regex = False)
    df.Place = df.Place.str.replace('Feb', '2', regex = False)
    df.Place = df.Place.str.replace('Mar', '3', regex = False)
    df.Place = df.Place.str.replace('-Nov', '', regex = False)
    df.Place = df.Place.str.replace('?', ' ', regex = False)
    df.loc[df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].index, 'Place'] = df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].Place.str.split(' ', expand = True)[0].str.strip('stnrdh') #Terrible
    df.loc[df[df.Place.fillna('').str.contains('[a-zA-Z]')].index, 'Place'] = '1'
    df.loc[df[~df.Place.fillna('1').str.isnumeric()].index, 'Place'] = np.nan
    return df

##### Name column

In [243]:
#Inspect
#df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].Name #Names that have characters other than a-z, spaces, apostrophe and dash.
#df[df.Name.str.contains("\s$")].Name
#df[df.Name.str.contains("- ")].Name

Series([], Name: Name, dtype: object)

In [241]:
#Fix
def fix_name(df):
    df.loc[12467, 'Name'] = 'Steven Melekiola'
    df.loc[2036, 'Name'] = "MAX O'TOOLE"
    df.loc[1491, 'Name'] = "Mark Shillington"
    df.loc[2706, 'Name'] = "MARK O'SHEA"
    df.loc[7688, 'Name'] = "Clair Angel"
    df.loc[3088, 'Name'] = "Rick Lewis"
    df.Name = df.Name.str.replace('?', '', regex = False)
    df.Name = df.Name.str.replace("'Sasha'", '', regex = False)
    df.Name = df.Name.str.replace('*', '', regex = False)
    df.Name = df.Name.str.replace('- ', '-', regex = False)
    df.Name = df.Name.str.replace(' -', '-', regex = False)
    df.Name = df.Name.str.replace('4th BP', '', regex = False)
    df.Name = df.Name.str.replace(' - u18', '', regex = False)
    df.Name = df.Name.str.replace('u18', '', regex = False)
    df.Name = df.Name.str.replace('??QLD', '', regex = False)
    df.Name = df.Name.str.replace('QLD', '', regex = False)
    df.Name = df.Name.str.replace(r'\([^)]*\)', '')
    df.Name = df.Name.str.replace(r'\"[^)]*\"', '')
    df.loc[df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].index, 'Name'] = df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].Name.str.split(', ').str[::-1].str.join(' ')
    df.Name = df.Name.str.replace('(\s{2})', '')
    df.Name = df.Name.str.strip()
    return df

##### TotalKg column

In [ ]:
#Inspect
df.Total.

### Execute

In [242]:
#One function
def fix_all(df):
    df = fix_sex(df)
    df = fix_class(df)
    df = fix_dob(df)
    df = fix_division(df)
    df = fix_wilks(df)
    df = fix_bdw(df)
    df = fix_place(df)
    df = fix_name(df)
    return df
df = fix_all(pd.read_csv("../../Data/Tidy_Full.csv", index_col = 0))

# Testing area

In [22]:
#opdf = pd.read_csv("../../Data/openpowerlifting-2021-03-31-ab223798.csv", index_col = 0)

In [23]:
df.columns

Index(['CompID', 'Name', 'Sex', 'Class', 'Division', 'BestBP', 'Total',
       'Place', 'BestSQ', 'BestDL', 'BDW', 'BP1', 'BP2', 'BP3', 'Wilks', 'SQ1',
       'SQ2', 'SQ3', 'DL1', 'DL2', 'DL3', 'DOB'],
      dtype='object')